# NYC Crime Reduction Project

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from sodapy import Socrata
from google.cloud import bigquery
from google.oauth2 import service_account

In [ ]:
# setting up the host name for the API endpoint
data_url = 'data.cityofnewyork.us'

In [ ]:
# create the client that points to the API endpoint
nyc_open_data_client = Socrata(data_url, app_token, timeout = 200)
print(f"nyc open data client name is: {nyc_open_data_client}")
print(f"nyc open data client data type is: {type(nyc_open_data_client)}")

nyc open data client name is: <sodapy.socrata.Socrata object at 0x7fdbc6b05880>
nyc open data client data type is: <class 'sodapy.socrata.Socrata'>


In [ ]:
# setting up big query credentials

credentials = service_account.Credentials.from_service_account_file(key_path,
                                                                    scopes=["https://www.googleapis.com/auth/cloud-platform"],)
bigquery_client = bigquery.Client(credentials = credentials,
                                 project = credentials.project_id)

print(f"bigquery client name is: {bigquery_client}")
print(f"bigquery client data type is: {type(bigquery_client)}")

bigquery client name is: <google.cloud.bigquery.client.Client object at 0x7fdbffc4e670>
bigquery client data type is: <class 'google.cloud.bigquery.client.Client'>


In [ ]:
# Get the total number of records in our the entire data set
total_record_count = nyc_open_data_client.get(data_set, select = "COUNT(*)")
print(f"total records in {data_set}: {total_record_count[0]['COUNT']}")

total records in 5uac-w243: 396978


In [ ]:
# Now, loop through target data set to pull all rows in chunks (we cannot pull all rows at once)
# AGAIN, UPDATE WHERE FILTER INSIDE BELOW FUNCTION

def extract_socrata_data(chunk_size = 2500,
                         data_set = data_set,
                         where = None):

    # measure time this function takes
    import time
    start_time = time.time()

    # get total number or records
    if where == None:
        total_records = int(nyc_open_data_client.get(data_set,
                                                     select= "COUNT(*)")[0]["COUNT"])
    else:
        total_records = int(nyc_open_data_client.get(data_set,
                                                     where = where,
                                                     select= "COUNT(*)")[0]["COUNT"])

    # start at 0, empty list for results
    start = 0
    results = []

    while True:

        if where == None:
            # fetch the set of records starting at 'start'
            results.extend(nyc_open_data_client.get(data_set,
                                                    offset = start,
                                                    limit = chunk_size))

        elif where != None:
            results.extend(nyc_open_data_client.get(data_set,
                                                    where = where,
                                                    offset = start,
                                                    limit = chunk_size))
        # update the starting record number
        start = start + chunk_size

        # if we have fetched all of the records (we have reached total_records), exit loop
        if (start > total_records):
            break

    # convert the list into a pandas data frame
    data = pd.DataFrame.from_records(results)

    end_time = time.time()
    print(f"function took {round(end_time - start_time, 1)} seconds")

    print(f"the shape of your dataframe is: {data.shape}")
    return data

In [ ]:
#CREATE DATAFRAME data HERE
df = extract_socrata_data(chunk_size = 2500,
                         data_set = data_set)

function took 140.6 seconds
the shape of your dataframe is: (396978, 41)


In [ ]:
#checking the names of our columns
df.columns

Index(['cmplnt_num', 'addr_pct_cd', 'boro_nm', 'cmplnt_fr_dt', 'cmplnt_fr_tm',
       'cmplnt_to_tm', 'crm_atpt_cptd_cd', 'hadevelopt', 'jurisdiction_code',
       'juris_desc', 'ky_cd', 'law_cat_cd', 'loc_of_occur_desc', 'ofns_desc',
       'parks_nm', 'patrol_boro', 'pd_cd', 'pd_desc', 'prem_typ_desc',
       'rpt_dt', 'station_name', 'susp_age_group', 'susp_race', 'susp_sex',
       'vic_age_group', 'vic_race', 'vic_sex', 'x_coord_cd', 'y_coord_cd',
       'latitude', 'longitude', 'lat_lon', 'geocoded_column',
       ':@computed_region_efsh_h5xi', ':@computed_region_f5dn_yrer',
       ':@computed_region_yeji_bk3q', ':@computed_region_92fq_4b7q',
       ':@computed_region_sbqj_enih', 'cmplnt_to_dt', 'housing_psa',
       'transit_district'],
      dtype='object')

In [ ]:
# ACTION REQUIRED
# edit the drop_columns list below to include all the columns you would like to drop
# then, run this cell to drop columns

drop_columns = ["addr_pct_cd", "hadevelopt", "jurisdiction_code", "juris_desc", "ky_cd",
                "loc_of_occur_desc", "ofns_desc", "parks_nm", "patrol_boro", "prem_typ_desc",
                "rpt_dt", "station_name", "x_coord_cd", "y_coord_cd", "lat_lon", "geocoded_column",
                ":@computed_region_efsh_h5xi", ":@computed_region_f5dn_yrer", ":@computed_region_yeji_bk3q",
                ":@computed_region_92fq_4b7q", ":@computed_region_sbqj_enih", "housing_psa", "transit_district"]

for column in drop_columns:
    try:
        df.drop(column, axis = 1, inplace = True)
    except:
        print(f"unable to drop {column}")

print(f"columns left in dataframe: {df.columns}")

columns left in dataframe: Index(['cmplnt_num', 'boro_nm', 'cmplnt_fr_dt', 'cmplnt_fr_tm', 'cmplnt_to_tm',
       'crm_atpt_cptd_cd', 'law_cat_cd', 'pd_cd', 'pd_desc', 'susp_age_group',
       'susp_race', 'susp_sex', 'vic_age_group', 'vic_race', 'vic_sex',
       'latitude', 'longitude', 'cmplnt_to_dt'],
      dtype='object')


In [ ]:
# creating and running a function to ceate data profiling dataframe

def create_data_profiling_df(data):

    # create an empty dataframe to gather information about each column
    data_profiling_df = pd.DataFrame(columns = ["column_name",
                                                "column_type",
                                                "unique_values",
                                                "duplicate_values",
                                                "null_values",
                                                "non_null_values"])

    # loop through each column to add rows to the data_profiling_df dataframe
    for column in data.columns:

        info_dict = {}

        try:
            info_dict["column_name"] = column
            info_dict["column_type"] = data[column].dtypes
            info_dict["unique_values"] = len(data[column].unique())
            info_dict["duplicate_values"] = data[column].count() - len(data[column].dropna().unique())
            info_dict["null_values"] = data[column].isna().sum()
            info_dict["non_null_values"] = data[column].count()

        except:
            print(f"unable to read column: {column}, you may want to drop this column")

        data_profiling_df = data_profiling_df.append(info_dict, ignore_index=True)

    data_profiling_df.sort_values(by = ['unique_values', "non_null_values"],
                                  ascending = [False, False],
                                  inplace=True)

    return data_profiling_df

In [ ]:

data_profiling_df = create_data_profiling_df(data = df)

In [ ]:
data_profiling_df

,column_name,column_type,unique_values,duplicate_values,null_values,non_null_values
0,cmplnt_num,object,396865,113,0,396978
16,longitude,object,57762,339208,9,396969
15,latitude,object,57387,339583,9,396969
2,cmplnt_fr_dt,object,1785,395193,0,396978
4,cmplnt_to_tm,object,1441,395537,0,396978
3,cmplnt_fr_tm,object,1440,395538,0,396978
17,cmplnt_to_dt,object,1226,366360,29393,367585
7,pd_cd,object,361,396178,440,396538
8,pd_desc,object,351,396627,0,396978
12,vic_age_group,object,24,396954,0,396978


In [ ]:
#Creating the suspect dimension, first we copy the df
SuspectDim = df.copy()

In [ ]:
#Now, subset for only the wanted columns in the dimension
SuspectDim = SuspectDim[["susp_sex", "susp_age_group", "susp_race"]]

In [ ]:
# Now, add SUSP_ID as a surrogate key
SuspectDim.insert(0, 'susp_id', range(1, 1 + len(SuspectDim)))
SuspectDim.head()

,susp_id,susp_sex,susp_age_group,susp_race
0,1,M,25-44,BLACK
1,2,U,25-44,UNKNOWN
2,3,(null),(null),(null)
3,4,(null),(null),(null)
4,5,(null),(null),(null)


In [ ]:
# Creating the Victim Dimension, first we copy the df
VictimDim = df.copy()

In [ ]:
#Now, subset for only the wanted columns in the dimension
VictimDim = VictimDim[["vic_sex", "vic_age_group", "vic_race"]]

In [ ]:
# Now, add VIC_ID as a surrogate key
VictimDim.insert(0, 'vic_id', range(1, 1 + len(VictimDim)))
VictimDim.head()

,vic_id,vic_sex,vic_age_group,vic_race
0,1,D,UNKNOWN,UNKNOWN
1,2,F,<18,BLACK
2,3,M,18-24,ASIAN / PACIFIC ISLANDER
3,4,M,45-64,WHITE
4,5,M,65+,BLACK


In [ ]:
# Creating the date dimension, first, copy the entire table
DateDim = df.copy()

In [ ]:
#Now, subset for only the wanted columns in the dimension
DateDim  = DateDim[["cmplnt_fr_dt", "cmplnt_to_dt"]]

In [ ]:
# add a surrogate key
DateDim.insert(0, 'date_id', range(1000, 1000 + len(DateDim)))
DateDim.head()

,date_id,cmplnt_fr_dt,cmplnt_to_dt
0,1000,2022-07-11T00:00:00.000,NaN
1,1001,2021-01-01T00:00:00.000,2022-09-07T00:00:00.000
2,1002,2021-12-24T00:00:00.000,2022-01-03T00:00:00.000
3,1003,2015-06-15T00:00:00.000,2022-07-13T00:00:00.000
4,1004,2018-01-01T00:00:00.000,2021-04-14T00:00:00.000


In [ ]:
#Creating time dimension
TimeDim = df.copy()

In [ ]:
TimeDim  = TimeDim[["cmplnt_fr_tm", "cmplnt_to_tm"]]

In [ ]:
# add a surrogate key
TimeDim.insert(0, 'time_id', range(1000, 1000 + len(TimeDim)))
TimeDim.head()

,time_id,cmplnt_fr_tm,cmplnt_to_tm
0,1000,09:09:00,(null)
1,1001,12:00:00,12:00:00
2,1002,08:00:00,08:00:00
3,1003,00:01:00,17:00:00
4,1004,09:00:00,23:59:00


In [ ]:
# Creating the offense dimension, first, copy the entire table
OffenseDim = df.copy()

In [ ]:
#Now, subset for only the wanted columns in the dimension
OffenseDim = OffenseDim[['law_cat_cd', 'pd_cd', 'pd_desc']]

In [ ]:
# Now, add offense_id as a surrogate key
OffenseDim.insert(0, 'Offense_id', range(100, 100 + len(OffenseDim)))
OffenseDim.head()

,Offense_id,law_cat_cd,pd_cd,pd_desc
0,100,FELONY,361,"ROBBERY,BANK"
1,101,FELONY,155,RAPE 2
2,102,FELONY,424,"LARCENY,GRAND BY CREDIT CARD ACCT COMPROMISE-E..."
3,103,FELONY,739,"FRAUD,UNCLASSIFIED-FELONY"
4,104,FELONY,739,"FRAUD,UNCLASSIFIED-FELONY"


In [ ]:
#Creating the location dimension, first, copy the entire table
LocationDim = df.copy()

In [ ]:
#Now, subset for only the wanted columns in the dimension
LocationDim = LocationDim[['latitude', 'longitude', 'boro_nm']]

In [ ]:
# Now, add location_id as a surrogate key
LocationDim.insert(0, 'Location_id', range(111000, 111000 + len(LocationDim)))
LocationDim.head()

,Location_id,latitude,longitude,boro_nm
0,111000,40.833567,-73.861118,BRONX
1,111001,40.6488507469884,-73.951016510623,BROOKLYN
2,111002,40.619768,-74.08407,STATEN ISLAND
3,111003,40.749791,-73.893792,QUEENS
4,111004,40.671636,-73.863359,BROOKLYN


In [ ]:
#Merging the original df with all the dimensions created to make one large dataset
#We treid to ujse the df.merge but got a memory issue
#Now using pandas concatenate to merge the file
df1 = pd.concat([df, DateDim.date_id, TimeDim.time_id, VictimDim.vic_id, SuspectDim.susp_id,
                           OffenseDim.Offense_id, LocationDim.Location_id],axis = 1)

In [ ]:
df1.head()

,cmplnt_num,boro_nm,cmplnt_fr_dt,cmplnt_fr_tm,cmplnt_to_tm,crm_atpt_cptd_cd,law_cat_cd,pd_cd,pd_desc,susp_age_group,...,vic_sex,latitude,longitude,cmplnt_to_dt,date_id,time_id,vic_id,susp_id,Offense_id,Location_id
0,247853895,BRONX,2022-07-11T00:00:00.000,09:09:00,(null),COMPLETED,FELONY,361,"ROBBERY,BANK",25-44,...,D,40.833567,-73.861118,NaN,1000,1000,1,1,100,111000
1,250722124,BROOKLYN,2021-01-01T00:00:00.000,12:00:00,12:00:00,COMPLETED,FELONY,155,RAPE 2,25-44,...,F,40.6488507469884,-73.951016510623,2022-09-07T00:00:00.000,1001,1001,2,2,101,111001
2,239511545,STATEN ISLAND,2021-12-24T00:00:00.000,08:00:00,08:00:00,COMPLETED,FELONY,424,"LARCENY,GRAND BY CREDIT CARD ACCT COMPROMISE-E...",(null),...,M,40.619768,-74.08407,2022-01-03T00:00:00.000,1002,1002,3,3,102,111002
3,247992275,QUEENS,2015-06-15T00:00:00.000,00:01:00,17:00:00,COMPLETED,FELONY,739,"FRAUD,UNCLASSIFIED-FELONY",(null),...,M,40.749791,-73.893792,2022-07-13T00:00:00.000,1003,1003,4,4,103,111003
4,239602232,BROOKLYN,2018-01-01T00:00:00.000,09:00:00,23:59:00,COMPLETED,FELONY,739,"FRAUD,UNCLASSIFIED-FELONY",(null),...,M,40.671636,-73.863359,2021-04-14T00:00:00.000,1004,1004,5,5,104,111004


In [ ]:
#Now, we create the fact table (CrimeComplaints)
CrimeComplaints_fact = df1[["crm_atpt_cptd_cd", "date_id", "time_id", "vic_id", "susp_id",
                           "Offense_id", "Location_id"]]
CrimeComplaints_fact.head()

,crm_atpt_cptd_cd,date_id,time_id,vic_id,susp_id,Offense_id,Location_id
0,COMPLETED,1000,1000,1,1,100,111000
1,COMPLETED,1001,1001,2,2,101,111001
2,COMPLETED,1002,1002,3,3,102,111002
3,COMPLETED,1003,1003,4,4,103,111003
4,COMPLETED,1004,1004,5,5,104,111004


In [ ]:
#created a function to load dataframes to BigQuery

def load_table_to_bigquery(df,
                          table_name,
                          dataset_id):

    dataset_id = dataset_id

    dataset_ref = bigquery_client.dataset(dataset_id)
    job_config = bigquery.LoadJobConfig()
    job_config.autodetect = True
    job_config.write_disposition = "WRITE_TRUNCATE"

    upload_table_name = f"{dataset_id}.{table_name}"

    load_job = bigquery_client.load_table_from_dataframe(df,
                                                upload_table_name,
                                                job_config = job_config)

    print(f"completed job {load_job}")

In [ ]:
#1st loading CrimeComplaints_fact table to BigQuery

load_table_to_bigquery(df = CrimeComplaints_fact,
                          table_name = "CrimeComplaints_fact",
                          dataset_id = dataset_id)

completed job LoadJob<project=pro-bruin-361120, location=US, id=c172b21d-cb92-4858-96f1-fb0859a8136a>


In [ ]:
#loading SuspectDim table to BigQuery

load_table_to_bigquery(df = SuspectDim,
                          table_name = "SuspectDim",
                          dataset_id = dataset_id)

completed job LoadJob<project=pro-bruin-361120, location=US, id=5c3ccd63-26d7-4d11-b257-cbc4508e22cf>


In [ ]:
#loading VictimDim table to BigQuery

load_table_to_bigquery(df = VictimDim,
                          table_name = "VictimDim",
                          dataset_id = dataset_id)

completed job LoadJob<project=pro-bruin-361120, location=US, id=20f08858-67e4-40ba-bc05-b901768a941e>


In [ ]:
#loading DateDim table to BigQuery

load_table_to_bigquery(df = DateDim,
                          table_name = "DateDim",
                          dataset_id = dataset_id)

completed job LoadJob<project=pro-bruin-361120, location=US, id=98e4c2a6-2967-4776-8b6e-984b8eecf5b4>


In [ ]:
#loading TimeDim table to BigQuery

load_table_to_bigquery(df = TimeDim,
                          table_name = "TimeDim",
                          dataset_id = dataset_id)

completed job LoadJob<project=pro-bruin-361120, location=US, id=8124484c-13b6-4fbc-ae65-322bff2b0d70>


In [ ]:
#loading OffenseDim table to BigQuery

load_table_to_bigquery(df = OffenseDim,
                          table_name = "OffenseDim",
                          dataset_id = dataset_id)

completed job LoadJob<project=pro-bruin-361120, location=US, id=618f2536-6244-4295-8960-73360e910d54>


In [ ]:
#loading LocationDim table to BigQuery

load_table_to_bigquery(df = LocationDim,
                          table_name = "LocationDim",
                          dataset_id = dataset_id)

completed job LoadJob<project=pro-bruin-361120, location=US, id=24540c54-4fc7-4364-a27c-6cbf04f22b26>


# Done: Loaded all Dimensions and Tables to BigQuery